In [ ]:
pip install beautifulsoup4


In [ ]:
pip install --upgrade mysql-connector-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 51.1 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import datetime
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import re
from google.colab import output
output.enable_custom_widget_manager()
import mysql.connector

In [ ]:
#SHOPPING ITAQUERA

def process_phone_number(phone):
    numbers_only = re.sub(r'[^\d]', '', phone)

    if not numbers_only:
        return None

    if numbers_only.startswith('11'):
        if len(numbers_only) == 11 and numbers_only[2] == '9':
            return f'({numbers_only[:2]}) {numbers_only[2:7]}-{numbers_only[7:]}'
        return None

    elif len(numbers_only) == 9 and numbers_only[0] == '9':
        return f'(11) {numbers_only[:5]}-{numbers_only[5:]}'
    elif len(numbers_only) == 8 and numbers_only[0] == '9':
        return f'(11) {numbers_only}-'

    return None


def scrape_loja_info(loja_div):
    try:
        loja = loja_div.find('h4')
        nome_loja = loja.text.strip() if loja else "Nome não encontrado"

        segmento = loja_div.find('span', class_='segmento')
        segmento = segmento.text.strip() if segmento else "Segmento não encontrado"

        wrap_info = loja_div.find_next_sibling('div', class_='wrapInfo')
        telefone = "Telefone não encontrado"

        if wrap_info:
            info_divs = wrap_info.find_all('div', class_='info')
            for info_div in info_divs:
                tel_link = info_div.find('a')
                if tel_link and 'tel:' in tel_link.get('href', ''):
                    numero_processado = process_phone_number(tel_link.text.strip())
                    if numero_processado:
                        telefone = numero_processado
                    break

        return {
            "nome": nome_loja,
            "segmento": segmento,
            "telefone": telefone,
            "shopping": "Itaquera"
        }

    except Exception as e:
        print(f"Erro ao extrair informações da loja {nome_loja}: {str(e)}")
        return None

def inserir_no_banco(conexao, loja_info):
    try:
        cursor = conexao.cursor()
        sql = """
        INSERT INTO lojas (loja, departamento, telefone, shopping)
        VALUES (%s, %s, %s, %s)
        """
        valores = (
            loja_info["nome"],
            loja_info["segmento"],
            loja_info["telefone"],
            loja_info["shopping"]
        )
        cursor.execute(sql, valores)
        conexao.commit()
        print(f"Dados inseridos com sucesso: {loja_info['nome']}")
    except Exception as e:
        print(f"Erro ao inserir dados no banco: {str(e)}")

def teste_scraping(url):
    try:
        conexao = mysql.connector.connect(
            host="74.207.244.188",
            user="hackday",
            password="wvtodoz08",
            database="hackday",
            auth_plugin="mysql_native_password"
        )

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        lojas_items = soup.find_all('li', class_='item')

        print(f"Encontradas {len(lojas_items)} lojas")
        celulares_encontrados = 0

        for loja_item in lojas_items:
            titulo_div = loja_item.find('div', class_='titulo')
            if titulo_div:
                info = scrape_loja_info(titulo_div)
                if info and info['telefone'] != "Telefone não encontrado":
                    celulares_encontrados += 1
                    print("\nInformações da loja:")
                    print(f"loja: {info['nome']}")
                    print(f"departamento: {info['segmento']}")
                    print(f"telefone: {info['telefone']}")
                    print(f"shopping: {info['shopping']}")

                    inserir_no_banco(conexao, info)

        print(f"\nTotal de celulares encontrados: {celulares_encontrados}")

    except Exception as e:
        print(f"Erro ao acessar a página: {str(e)}")

    finally:
        if 'conexao' in locals() and conexao.is_connected():
            conexao.close()
            print("Conexão com o banco encerrada.")

url = "https://www.shoppingitaquera.com.br/lojas"
teste_scraping(url)

Encontradas 247 lojas

Informações da loja:
loja: ALIANÇAS CS
departamento: Joias, relógios e bijuterias
telefone: (11) 97025-1038
shopping: Itaquera
Dados inseridos com sucesso: ALIANÇAS CS

Informações da loja:
loja: AMERICAN SHOES
departamento: Calçados
telefone: (11) 98904-9821
shopping: Itaquera
Dados inseridos com sucesso: AMERICAN SHOES

Informações da loja:
loja: ANGELA CALÇADOS
departamento: Calçados
telefone: (11) 99881-6700
shopping: Itaquera
Dados inseridos com sucesso: ANGELA CALÇADOS

Informações da loja:
loja: BRANDILI
departamento: Vestuário
telefone: (11) 98591-3191
shopping: Itaquera
Dados inseridos com sucesso: BRANDILI

Informações da loja:
loja: CAMISARIA FMW
departamento: Vestuário
telefone: (11) 93085-9913
shopping: Itaquera
Dados inseridos com sucesso: CAMISARIA FMW

Informações da loja:
loja: CASA DO CELULAR
departamento: Telefonia e acessórios
telefone: (11) 99003-3164
shopping: Itaquera
Dados inseridos com sucesso: CASA DO CELULAR

Informações da loja:
loja: 

In [ ]:
# SHIPPING CIDADE SÃO PAULO

def process_phone_number(phone):
    numbers_only = re.sub(r'[^\d]', '', phone)

    if not numbers_only:
        return None

    if numbers_only.startswith('11'):
        if len(numbers_only) == 11 and numbers_only[2] == '9':
            return f'({numbers_only[:2]}) {numbers_only[2:7]}-{numbers_only[7:]}'
        return None

    elif len(numbers_only) == 9 and numbers_only[0] == '9':
        return f'(11) {numbers_only[:5]}-{numbers_only[5:]}'
    elif len(numbers_only) == 8 and numbers_only[0] == '9':
        return f'(11) {numbers_only}-'

    return None


def scrape_loja_info(loja_div):
    try:
        # Nome da loja
        nome_loja = loja_div.find('h3', class_='notranslate')
        nome_loja = nome_loja.text.strip() if nome_loja else "Nome não encontrado"

        # Segmento (Serviços, por exemplo, poderia ser mais específico)
        segmento = loja_div.find('p')
        segmento = segmento.text.strip() if segmento else "Segmento não encontrado"

        # Telefone
        telefone = "Telefone não encontrado"
        tel_link = loja_div.find('a', href=re.compile(r'tel:'))

        if tel_link:
            numero_processado = process_phone_number(tel_link.text.strip())
            if numero_processado:
                telefone = numero_processado

        return {
            "nome": nome_loja,
            "segmento": segmento,
            "telefone": telefone,
            "shopping": "Cidade São Paulo"
        }

    except Exception as e:
        print(f"Erro ao extrair informações da loja: {str(e)}")
        return None


def inserir_no_banco(conexao, loja_info):
    try:
        cursor = conexao.cursor()
        sql = """
        INSERT INTO lojas (loja, departamento, telefone, shopping)
        VALUES (%s, %s, %s, %s)
        """
        valores = (
            loja_info["nome"],
            loja_info["segmento"],
            loja_info["telefone"],
            loja_info["shopping"]
        )
        cursor.execute(sql, valores)
        conexao.commit()
        print(f"Dados inseridos com sucesso: {loja_info['nome']}")
    except Exception as e:
        print(f"Erro ao inserir dados no banco: {str(e)}")


def teste_scraping(url):
    try:
        conexao = mysql.connector.connect(
            host="74.207.244.188",
            user="hackday",
            password="wvtodoz08",
            database="hackday",
            auth_plugin="mysql_native_password"
        )

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        # Modifique a busca para encontrar os itens de loja corretamente
        lojas_items = soup.find_all('div', class_='text')

        print(f"Encontradas {len(lojas_items)} lojas")
        celulares_encontrados = 0

        for loja_item in lojas_items:
            info = scrape_loja_info(loja_item)
            if info and info['telefone'] != "Telefone não encontrado":
                celulares_encontrados += 1
                print("\nInformações da loja:")
                print(f"loja: {info['nome']}")
                print(f"departamento: {info['segmento']}")
                print(f"telefone: {info['telefone']}")
                print(f"shopping: {info['shopping']}")

                inserir_no_banco(conexao, info)

        print(f"\nTotal de celulares encontrados: {celulares_encontrados}")

    except Exception as e:
        print(f"Erro ao acessar a página: {str(e)}")

    finally:
        if 'conexao' in locals() and conexao.is_connected():
            conexao.close()
            print("Conexão com o banco encerrada.")

url = "https://shoppingcidadesp.com.br/lojas"
teste_scraping(url)

Encontradas 168 lojas

Informações da loja:
loja: Acium
departamento: Joalheria
telefone: (11) 96374-9292
shopping: Cidade São Paulo
Dados inseridos com sucesso: Acium

Informações da loja:
loja: Altino Espumantes
departamento: Bebidas
telefone: (11) 99283-4104
shopping: Cidade São Paulo
Dados inseridos com sucesso: Altino Espumantes

Informações da loja:
loja: Animale
departamento: Moda Feminina
telefone: (11) 99647-8767
shopping: Cidade São Paulo
Dados inseridos com sucesso: Animale

Informações da loja:
loja: Arezzo
departamento: Calçados
telefone: (11) 91306-8446
shopping: Cidade São Paulo
Dados inseridos com sucesso: Arezzo

Informações da loja:
loja: Baked Potato
departamento: Alimentação
telefone: (11) 98951-5352
shopping: Cidade São Paulo
Dados inseridos com sucesso: Baked Potato

Informações da loja:
loja: Bennemann
departamento: Malas e bolsas
telefone: (11) 91605-0068
shopping: Cidade São Paulo
Dados inseridos com sucesso: Bennemann

Informações da loja:
loja: Biscoitê
depar

In [ ]:
!pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.0 MB/s eta 0:00:00


In [ ]:
# COMPLEXO TATUAPÉ

import requests
import json
import mysql.connector
from mysql.connector import Error

url = "https://sal.madnezz.com.br/api/site/json/loja.asp?tipo=1,2,3&shopping_id=19&full=true&jsoncallback=jQuery111103522873809313347_1737047921158&_=1737047921159"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

def conectar_banco():
    try:
        connection = mysql.connector.connect(
            host='74.207.244.188',
            user='hackday',
            password='wvtodoz08',
            database='hackday'
        )
        return connection
    except Error as e:
        print(f"Erro de conexão com o banco de dados: {e}")
        return None

def inserir_dados_loja(connection, shopping, nome, email, whatsapp, ramo):
    try:
        cursor = connection.cursor()
        query = "INSERT INTO lojas (shopping, loja, email, telefone, departamento) VALUES (%s, %s, %s, %s, %s)"
        data = (shopping, nome, email, whatsapp, ramo)
        cursor.execute(query, data)
        connection.commit()
        print(f"Loja '{nome}' inserida com sucesso.")
    except Error as e:
        print(f"Erro ao inserir dados no banco: {e}")

response = requests.get(url, headers=headers)

if response.status_code == 200:
    content = response.text.strip()
    if content.startswith("/**/jQuery"):
        content = content.split("(", 1)[-1].rsplit(")", 1)[0]

    data = json.loads(content)

    connection = conectar_banco()

    if connection:
        lojas = data.get("loja", [])
        for loja in lojas:
            shopping = "Complexo Tatuapé"
            nome = loja.get("nome", "N/A")
            ramo = loja.get("ramo_nome", "N/A")
            email = loja.get("loja_email", "N/A")
            whatsapp = loja.get("whatsapp", "N/A")

            if whatsapp and whatsapp != "N/A":
                inserir_dados_loja(connection, shopping, nome, email, whatsapp, ramo)

        connection.close()

    print(response.text)
else:
    print(f"Erro na requisição: {response.status_code}")

Loja ' L'Occitane Au Brésil' inserida com sucesso.
Loja 'ACCESS QUALITY PREMIUM' inserida com sucesso.
Loja 'ACIUM' inserida com sucesso.
Loja 'ÁGUA DE CHEIRO' inserida com sucesso.
Loja 'ALIANÇAS CS' inserida com sucesso.
Loja 'ALIANÇAS LUPATELLI ' inserida com sucesso.
Loja 'AMÁVIA' inserida com sucesso.
Loja 'AMOR AOS PEDAÇOS' inserida com sucesso.
Loja 'AMORA' inserida com sucesso.
Loja 'ANACAPRI' inserida com sucesso.
Loja 'ART WALK' inserida com sucesso.
Loja 'AUTHENTIC FEET' inserida com sucesso.
Loja 'BACIO DI LATTE' inserida com sucesso.
Loja 'Bad Cat' inserida com sucesso.
Loja 'Bad Cat' inserida com sucesso.
Loja 'BADDER' inserida com sucesso.
Loja 'Baducco Empório Panettone' inserida com sucesso.
Loja 'BAKED POTATO' inserida com sucesso.
Loja 'BARBESHOP BELO'SCA' inserida com sucesso.
Loja 'BEAUTY LAND                   ' inserida com sucesso.
Loja 'BELLA ACTO COSMETICOS' inserida com sucesso.
Loja 'BELLA VIDA' inserida com sucesso.
Loja 'BEM QUERER - AQUI TEM NATURA' inser

In [ ]:
# BOURBON SHOPPING

import requests
import mysql.connector
import time

base_url = "https://www.bourbonshopping.com.br/bourbon-shopping/lojas"

# Parâmetros da requisição
page = 1
page_size = 18
has_more_pages = True

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:115.0) Gecko/20100101 Firefox/115.0",
    "Accept": "application/json",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Referer": "https://www.bourbonshopping.com.br",
    "Origin": "https://www.bourbonshopping.com.br",
    "Host": "www.bourbonshopping.com.br"
}

connection = mysql.connector.connect(
    host="74.207.244.188",
    user="hackday",
    password="wvtodoz08",
    database="hackday"
)

cursor = connection.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS lojas (
        loja VARCHAR(255),
        telefone VARCHAR(255),
        departamento VARCHAR(255),
        shopping VARCHAR(255),
        PRIMARY KEY (loja, telefone)
    )
""")
connection.commit()

while has_more_pages:
    params = {
        "page": page,
        "pageSize": page_size,
        "codigoSapRe": "",
        "searchLoja": "",
        "atividade": ""
    }

    try:
        print(f"\nFazendo requisição para página {page}")
        time.sleep(2)

        response = requests.get(base_url, params=params, headers=headers)
        print(f"Status code: {response.status_code}")

        if response.status_code != 200:
            print(f"Erro na requisição: {response.status_code}")
            print(f"Conteúdo da resposta: {response.text}")
            break

        try:
            data = response.json()
        except ValueError as e:
            print(f"Erro ao decodificar JSON na página {page}")
            print(f"Conteúdo bruto da resposta: {response.text}")
            print(f"Erro específico: {str(e)}")
            break

        lojas = data.get("items", [])
        print(f"Número de lojas encontradas: {len(lojas)}")

        for loja in lojas:
            nome = loja.get("titulo")
            telefone = loja.get("telefone_site")
            departamento = loja.get("atividade") or "Não informado"

            if (nome and telefone and telefone.isdigit() and
                len(telefone) == 11 and telefone[2] == '9'):
                try:
                    cursor.execute(
                        "INSERT INTO lojas (loja, telefone, departamento, shopping) VALUES (%s, %s, %s, %s)",
                        (nome, telefone, departamento, "Bourbon")
                    )
                    connection.commit()
                    print(f"Loja inserida com sucesso: {nome} - {telefone}")
                except mysql.connector.IntegrityError as e:
                    print(f"Erro ao inserir dados: {e}")
            else:
                print(f"Telefone não é celular ou dados inválidos: {nome} - {telefone}")

        has_more_pages = data.get("next_page_url") is not None
        print(f"Tem mais páginas? {has_more_pages}")
        page += 1

    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        break

cursor.execute("SELECT COUNT(*) FROM lojas")
total_lojas = cursor.fetchone()[0]
print(f"\nTotal de lojas no banco: {total_lojas}")

cursor.execute("SELECT * FROM lojas LIMIT 5")
print("\nPrimeiras 5 lojas no banco:")
for row in cursor.fetchall():
    print(row)

# Fechar a conexão
cursor.close()
connection.close()


Fazendo requisição para página 1
Status code: 200
Número de lojas encontradas: 18
Telefone não é celular ou dados inválidos: 5ASEC - 5132736167
Loja inserida com sucesso: A OCULISTA - 11966539511
Telefone não é celular ou dados inválidos: ABBRACCIO – CUCINA ITALIANA - 1150434082
Loja inserida com sucesso: ACIUM - 11949745887
Loja inserida com sucesso: ACQUAZERO ECO CAR WASH - 11998766888
Telefone não é celular ou dados inválidos: ADCOS DERMOCOSMETICOS - 1142802642
Loja inserida com sucesso: ADIDAS ORIGINALS - 11911859828
Telefone não é celular ou dados inválidos: ADUANA - 5137811271
Loja inserida com sucesso: AJR MANIA DE LER BOOK HALL - 51991922765
Loja inserida com sucesso: ALCHI - 54994231881
Telefone não é celular ou dados inválidos: ALECRIM PRESENTES - 5130280902
Telefone não é celular ou dados inválidos: ALECRIM PRESENTES - 5125007703
Telefone não é celular ou dados inválidos: ALECRIM PRESENTES - None
Telefone não é celular ou dados inválidos: ALECRIM PRESENTES - None
Telefone n

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import mysql.connector

def process_phone_number(phone):
    numbers_only = re.sub(r'[^\d]', '', phone)

    if not numbers_only:
        return None

    # Ignorar números fixos (8 dígitos que não começam com 9)
    if len(numbers_only) == 8 and not numbers_only.startswith('9'):
        return None

    # Validar número com DDD (11 dígitos)
    if len(numbers_only) == 11:
        # Verifica se é um DDD válido (10-99) e se começa com 9
        if numbers_only[0] in '1234567890' and numbers_only[1] in '1234567890' and numbers_only[2] == '9':
            return f'({numbers_only[:2]}) {numbers_only[2:7]}-{numbers_only[7:]}'
        return None

    # Validar celular sem DDD (9 dígitos)
    elif len(numbers_only) == 9 and numbers_only[0] == '9':
        # Assumir DDD 11 para manter compatibilidade com o comportamento anterior
        # quando não há DDD especificado
        return f'(11) {numbers_only[:5]}-{numbers_only[5:]}'

    return None

def scrape_loja_info(loja_div):
    try:
        nome_loja = loja_div.find('div', class_='wrapper').find('h3', class_='notranslate')
        nome_loja = nome_loja.text.strip() if nome_loja else "Nome não encontrado"

        wrapper = loja_div.find('div', class_='wrapper')
        segmento = wrapper.find('p')
        segmento = segmento.text.strip() if segmento else "Segmento não encontrado"

        telefone = "Telefone não encontrado"
        tel_paragraphs = wrapper.find_all('p')
        if len(tel_paragraphs) > 1:
            tel_link = tel_paragraphs[1].find('a', href=re.compile(r'tel:'))
            if tel_link:
                numero_processado = process_phone_number(tel_link.text.strip())
                if numero_processado:
                    telefone = numero_processado

        return {
            "nome": nome_loja,
            "segmento": segmento,
            "telefone": telefone,
            "shopping": "Shopping D"
        }

    except Exception as e:
        print(f"Erro ao extrair informações da loja: {str(e)}")
        return None

def inserir_no_banco(conexao, loja_info):
    try:
        cursor = conexao.cursor()
        sql = """
        INSERT INTO lojas (loja, departamento, telefone, shopping)
        VALUES (%s, %s, %s, %s)
        """
        valores = (
            loja_info["nome"],
            loja_info["segmento"],
            loja_info["telefone"],
            loja_info["shopping"]
        )
        cursor.execute(sql, valores)
        conexao.commit()
        print(f"Dados inseridos com sucesso: {loja_info['nome']}")
    except Exception as e:
        print(f"Erro ao inserir dados no banco: {str(e)}")

def teste_scraping(url):
    try:
        conexao = mysql.connector.connect(
            host="74.207.244.188",
            user="hackday",
            password="wvtodoz08",
            database="hackday",
            auth_plugin="mysql_native_password"
        )

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        lojas_items = soup.find_all('div', class_='inner')

        print(f"Encontradas {len(lojas_items)} lojas")
        celulares_encontrados = 0

        for loja_item in lojas_items:
            info = scrape_loja_info(loja_item)
            # Só processa se for um celular válido (não é "Telefone não encontrado")
            if info and info['telefone'] != "Telefone não encontrado":
                celulares_encontrados += 1
                print("\nInformações da loja:")
                print(f"Loja: {info['nome']}")
                print(f"Departamento: {info['segmento']}")
                print(f"Telefone: {info['telefone']}")
                print(f"Shopping: {info['shopping']}")

                inserir_no_banco(conexao, info)

        print(f"\nTotal de celulares encontrados: {celulares_encontrados}")

    except Exception as e:
        print(f"Erro ao acessar a página: {str(e)}")

    finally:
        if 'conexao' in locals() and conexao.is_connected():
            conexao.close()
            print("Conexão com o banco encerrada.")

url = "https://shoppingd.com.br/lojas"
teste_scraping(url)

Encontradas 114 lojas
Erro ao extrair informações da loja: 'NoneType' object has no attribute 'find'

Informações da loja:
Loja: Americanas
Departamento: Acessórios, Acessórios para celular, Brinquedos, Calçados, Eletrônicos, Livraria, Moda Infantil, Papelaria, Utilidades, Vestuário, Vestuário feminino, Vestuário masculino
Telefone: (11) 96313-4891
Shopping: Shopping D
Dados inseridos com sucesso: Americanas

Informações da loja:
Loja: BBurgers
Departamento: Alimentação, Fast Food
Telefone: (11) 99946-1255
Shopping: Shopping D
Dados inseridos com sucesso: BBurgers

Informações da loja:
Loja: Casa das Sapatilhas
Departamento: Calçados
Telefone: (11) 96154-4236
Shopping: Shopping D
Dados inseridos com sucesso: Casa das Sapatilhas

Informações da loja:
Loja: Chilli Beans
Departamento: Acessórios, Ótica
Telefone: (11) 97860-8059
Shopping: Shopping D
Dados inseridos com sucesso: Chilli Beans

Informações da loja:
Loja: Dionelly
Departamento: Entretenimento, Entretenimento infantil
Telefone:

In [ ]:
pip install selenium webdriver-manager


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 33.5 MB/s eta 0:00:00


In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import mysql.connector

def process_phone_number(phone):
    numbers_only = re.sub(r'[^\d]', '', phone)

    if not numbers_only:
        return None

    # Verificar o comprimento do número
    if len(numbers_only) == 11:
        # Verificar se é um número de celular (começa com 9 após o DDD)
        if numbers_only[2] == '9':
            return f'({numbers_only[:2]}) {numbers_only[2:7]}-{numbers_only[7:]}'
        return None

    # Validar celular sem DDD (9 dígitos)
    elif len(numbers_only) == 9 and numbers_only[0] == '9':
        return f'(11) {numbers_only[:5]}-{numbers_only[5:]}'  # Mantém 11 como DDD padrão quando não informado

    return None

def scrape_loja_info(loja_item):
    try:
        # Extrair nome da loja
        nome_loja = loja_item.find('h1', class_='c_white')
        nome_loja = nome_loja.text.strip() if nome_loja else "Nome não encontrado"

        # Encontrar a div que contém as informações
        info_div = loja_item.find('div', class_='box tal loja_info border_box')

        if info_div:
            # Extrair segmento/ramo
            ramo_p = info_div.find('p', class_='loja_telefone', text=re.compile('Ramo:'))
            if ramo_p and ramo_p.find_next('h3'):
                segmento = ramo_p.find_next('h3').text.strip()
            else:
                segmento = "Segmento não encontrado"

            # Extrair telefone
            telefone = "Telefone não encontrado"
            tel_p = info_div.find('p', class_='loja_telefone', text=re.compile('Telefone:'))
            if tel_p and tel_p.find_next('h3'):
                tel_h3 = tel_p.find_next('h3')
                for img in tel_h3.find_all('img'):
                    img.decompose()
                numero = tel_h3.text.strip()
                numero_processado = process_phone_number(numero)
                if numero_processado:
                    telefone = numero_processado

        return {
            "nome": nome_loja,
            "segmento": segmento,
            "telefone": telefone,
            "shopping": "Shopping Center 3"
        }

    except Exception as e:
        print(f"Erro ao extrair informações da loja: {str(e)}")
        return None

def inserir_no_banco(conexao, lojas_info):
    try:
        cursor = conexao.cursor()
        sql = """
        INSERT INTO lojas (loja, departamento, telefone, shopping)
        VALUES (%s, %s, %s, %s)
        """
        # Inserir múltiplas lojas de uma vez
        valores = [(loja["nome"], loja["segmento"], loja["telefone"], loja["shopping"]) for loja in lojas_info]

        cursor.executemany(sql, valores)
        conexao.commit()
        print(f"{len(lojas_info)} dados inseridos com sucesso.")
    except Exception as e:
        print(f"Erro ao inserir dados no banco: {str(e)}")

def teste_scraping(url):
    try:
        conexao = mysql.connector.connect(
            host="74.207.244.188",
            user="hackday",
            password="wvtodoz08",
            database="hackday",
            auth_plugin="mysql_native_password"
        )

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }

        print(f"Tentando acessar URL: {url}")
        response = requests.get(url, headers=headers, verify=True)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        lojas_items = soup.find_all('li', class_='pr')
        if not lojas_items:
            lojas_items = soup.find_all('div', class_='inner')
        if not lojas_items:
            lojas_items = soup.select('.listagem-lojas li')

        print(f"Encontradas {len(lojas_items)} lojas")

        lojas_info = []

        for loja_item in lojas_items:
            info = scrape_loja_info(loja_item)
            if info and info['telefone'] != "Telefone não encontrado":
                lojas_info.append(info)

        if lojas_info:
            inserir_no_banco(conexao, lojas_info)

    except Exception as e:
        print(f"Erro ao acessar a página: {str(e)}")

    finally:
        if 'conexao' in locals() and conexao.is_connected():
            conexao.close()
            print("Conexão com o banco encerrada.")

# URL atualizada para HTTPS
url = "https://www.shoppingcenter3.com.br/loja.asp"
teste_scraping(url)

Tentando acessar URL: https://www.shoppingcenter3.com.br/loja
Erro ao acessar a página: 404 Client Error: Not Found for url: https://www.shoppingcenter3.com.br/loja
Conexão com o banco encerrada.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep

# Inicialize o WebDriver (certifique-se de ter o ChromeDriver ou outro driver)
driver = webdriver.Chrome()

# Acesse a página do Shopping
driver.get("https://www.shoppingcenter3.com.br/loja.asp")

# Aguarde o carregamento da página
sleep(5)  # Pode ajustar esse tempo conforme necessário

# Encontre os elementos das lojas. Você pode ajustar o seletor para corresponder à estrutura real da página
lojas = driver.find_elements(By.CLASS_NAME, "nome-loja")  # Ajuste esse seletor conforme necessário

# Extraia e imprima as informações das lojas
for loja in lojas:
    nome_loja = loja.text
    print(f'Loja: {nome_loja}')

# Feche o navegador após a execução
driver.quit()





import requests

# URL da API que retorna dados das lojas (identificada no passo 1)
url = "URL_DA_API_AQUI"

# Parâmetros que a API pode precisar (verifique nas requisições do navegador)
params = {
    "shopping_id": 107,  # Exemplo de parâmetro
    # Adicione outros parâmetros conforme necessário
}

# Realiza a requisição à API
response = requests.get(url, params=params)

# Verifica se a requisição foi bem-sucedida
if response.status_code == 200:
    data = response.json()  # A resposta é JSON, então converta para um objeto Python
    for loja in data:
        print(f'Loja: {loja["loja_nome"]}')  # Ajuste conforme a estrutura da resposta
else:
    print("Erro ao buscar dados.")

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir
Stacktrace:
#0 0x5afe1d6967ca <unknown>
#1 0x5afe1d18e2f0 <unknown>
#2 0x5afe1d1c5063 <unknown>
#3 0x5afe1d1c18c6 <unknown>
#4 0x5afe1d20ebc9 <unknown>
#5 0x5afe1d20e216 <unknown>
#6 0x5afe1d202753 <unknown>
#7 0x5afe1d1cfbaa <unknown>
#8 0x5afe1d1d0dfe <unknown>
#9 0x5afe1d66138b <unknown>
#10 0x5afe1d665307 <unknown>
#11 0x5afe1d64de7c <unknown>
#12 0x5afe1d665ec7 <unknown>
#13 0x5afe1d63224f <unknown>
#14 0x5afe1d6852f8 <unknown>
#15 0x5afe1d6854c0 <unknown>
#16 0x5afe1d695646 <unknown>
#17 0x781714f44ac3 <unknown>
